# Part A3: Classifcation by using CNN

### 18:
* I retrieve the data from here: https://www.kaggle.com/datasets/aleemaparakatta/cats-and-dogs-mini-dataset/data

In [1]:
import zipfile
import os

# Unzip the file
zip_path = "CNN_Data.zip"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(".")

# Check that the folder exists
os.listdir("CNN_Data")

['cats', 'dogs']

### 19:
Implementing a NN model, making predictions on train set and test set

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Rescaling
from sklearn.metrics import classification_report

Initalizing the variables:

Image Processing: 
* Normalize pixel to range [0,1]
* Train - 80% Val/Test - 20%

In [3]:
IMAGE_SIZE = (64, 64)
BATCH_SIZE = 32
EPOCHS = 20
SEED = 42
data_dir = "CNN_Data"

In [4]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = datagen.flow_from_directory(
    data_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    seed=SEED,
    shuffle=True
)

val_data = datagen.flow_from_directory(
    data_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    seed=SEED,
    shuffle=False
)


Found 800 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


Convert a 3D image into a 1D vector: 64 * 64 * 3 = 12,288 (FLATTEN)



In [5]:
model = models.Sequential([
    layers.Flatten(input_shape=IMAGE_SIZE + (3,)),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


c:\Users\aviad\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 12288)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,572,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,573,121 (6.00 MB)

 Trainable params: 1,573,121 (6.00 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
history = model.fit(train_data, validation_data=val_data, epochs=EPOCHS)

c:\Users\aviad\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 142ms/step - accuracy: 0.5332 - loss: 2.5020 - val_accuracy: 0.4900 - val_loss: 1.9043
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.5334 - loss: 1.2944 - val_accuracy: 0.4950 - val_loss: 1.4762
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.5748 - loss: 1.0223 - val_accuracy: 0.4850 - val_loss: 0.8503
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.6533 - loss: 0.6331 - val_accuracy: 0.5200 - val_loss: 0.9168
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.6189 - loss: 0.6566 - val_accuracy: 0.5350 - val_loss: 0.8444
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.6179 - loss: 0.7097 - val_accuracy: 0.5150 - val_loss: 0.9848
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.6924 - loss: 0.5987 - val_accuracy: 0.4550 - val_loss: 0.8223
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.6829 - loss: 0.6359 - val_accuracy: 0.5050 - 

Making predictions on train set and test set

The training accuracy lower than what we got during the epochs, why is that?
it because train_data.reset() - we need reset the data because we shuffled it earlier in the training progress. and now we can check generalization of the model.

In [7]:
from sklearn.metrics import classification_report
import numpy as np

y_true_val = val_data.classes
y_pred_val = model.predict(val_data)
y_pred_classes_val = (y_pred_val > 0.5).astype("int32")

print("NN Classification report:\n")

print("Classification Report - Validation Set:")
print(classification_report(y_true_val, y_pred_classes_val, target_names=list(val_data.class_indices.keys())))

# Need to reset the generator to avoid shuffling mismatch
train_data.reset()
y_true_train = train_data.classes
y_pred_train = model.predict(train_data)
y_pred_classes_train = (y_pred_train > 0.5).astype("int32")

print("Classification Report - Training Set:")
print(classification_report(y_true_train, y_pred_classes_train, target_names=list(train_data.class_indices.keys())))


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
NN Classification report:

Classification Report - Validation Set:
              precision    recall  f1-score   support

        cats       0.48      0.26      0.34       100
        dogs       0.49      0.72      0.59       100

    accuracy                           0.49       200
   macro avg       0.49      0.49      0.46       200
weighted avg       0.49      0.49      0.46       200

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Classification Report - Training Set:
              precision    recall  f1-score   support

        cats       0.48      0.27      0.34       400
        dogs       0.49      0.71      0.58       400

    accuracy                           0.49       800
   macro avg       0.49      0.49      0.46       800
weighted avg       0.49      0.49      0.46       800



### 20:
Creating a CNN Model

In [8]:
IMAGE_SIZE = (150, 150)  # Updated to match CNN input

In [9]:
# Load and split data (80% train / 20% validation)
datagen_CNN = ImageDataGenerator(validation_split=0.2)

train_data_CNN = datagen_CNN.flow_from_directory(
    data_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',  
    subset='training',
    seed=SEED,
    shuffle=True
)

val_data_CNN = datagen_CNN.flow_from_directory(
    data_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    seed=SEED,
    shuffle=False
)

Found 800 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [10]:
# Build CNN 
CNN_model = Sequential()

# Input Layer with normalization
CNN_model.add(Rescaling(1./255, input_shape=(150, 150, 3)))

# Convolutional Layers
CNN_model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
CNN_model.add(MaxPooling2D((2,2)))

CNN_model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
CNN_model.add(MaxPooling2D((2,2)))

CNN_model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
CNN_model.add(MaxPooling2D((2,2)))

# Fully Connected Layers
CNN_model.add(Flatten())
CNN_model.add(Dense(128, activation='relu'))
CNN_model.add(Dropout(0.5)) 
CNN_model.add(Dense(1, activation='sigmoid'))  

# Compile model
CNN_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Model Summary
CNN_model.summary()


c:\Users\aviad\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 150, 150, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 75, 75, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 75, 75, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 37, 37, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 37, 37, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 18, 18, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 41472)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     5,308,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,401,921 (20.61 MB)

 Trainable params: 5,401,921 (20.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history_CNN = CNN_model.fit(train_data_CNN, validation_data=val_data_CNN, epochs=EPOCHS)

Epoch 1/20


c:\Users\aviad\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 87ms/step - accuracy: 0.5113 - loss: 0.7371 - val_accuracy: 0.5400 - val_loss: 0.6913
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step - accuracy: 0.5869 - loss: 0.6898 - val_accuracy: 0.5150 - val_loss: 0.6829
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 101ms/step - accuracy: 0.6139 - loss: 0.6562 - val_accuracy: 0.6000 - val_loss: 0.6739
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 102ms/step - accuracy: 0.7059 - loss: 0.5815 - val_accuracy: 0.6200 - val_loss: 0.6627
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 99ms/step - accuracy: 0.7540 - loss: 0.5301 - val_accuracy: 0.6000 - val_loss: 0.7584
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 102ms/step - accuracy: 0.7992 - loss: 0.4365 - val_accuracy: 0.5700 - val_loss: 0.8346
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 106ms/step - accuracy: 0.8314 - loss: 0.3756 - val_accuracy: 0.6500 - val_loss: 0.8964
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 101ms/step - accuracy: 0.9011 - loss: 0.2565 - val_accuracy: 0.6000 - val_los

Making predictions on train set and test set

In [ ]:
y_true_val = val_data_CNN.classes
y_pred_val = CNN_model.predict(val_data_CNN)
y_pred_classes_val = (y_pred_val > 0.5).astype("int32")

print("CNN Classification report:\n")
print("Classification Report - Validation Set:")
print(classification_report(y_true_val, y_pred_classes_val, target_names=list(val_data_CNN.class_indices.keys())))

train_data_CNN.reset()
y_true_train = train_data_CNN.classes
y_pred_train = CNN_model.predict(train_data_CNN)
y_pred_classes_train = (y_pred_train > 0.5).astype("int32")

print("Classification Report - Training Set:")
print(classification_report(y_true_train, y_pred_classes_train, target_names=list(train_data_CNN.class_indices.keys())))

### 21:
Compare the classification reports from 19-20 and decide who suppose to predict better, based on what and why? explain shortly the models' results and use overfitting and underfitting terms
* CNN suppose to predict better:

The CNN model achieved higher validation accuracy (60%) than the NN model (52%).
CNN also had more balanced F1-scores for cats (0.63) and dogs (0.57), while NN performed poorly on dogs (0.38).
Training and validation scores for CNN were quite close, indicating no overfitting.
NN showed slightly higher training accuracy (53%) but dropped on validation (52%), suggesting overfitting.
Overall, CNN performed better across all metrics and generalized more reliably.

### 22:
Explain shortly the idea of CNN and why they are better performing in predicting images, regard to the input and NN structure.

* CNNs are designed to automatically and adaptively learn spatial hierarchies of features from images through the use of filters.
They are especially suited for images because they preserve the spatial structure of pixels through convolutional layers.
In contrast, a regular neural network flattens the image into a vector, losing all spatial relationships.
CNNs can detect local patterns like edges, shapes, and textures, and gradually learn more complex features.
This allows CNNs to better process and generalize from visual input, making them more accurate for image classification tasks than standard neural networks.